In [2]:
import numpy as np
import pandas as pd

In [2]:
spread_prob_df = pd.concat([pd.read_csv('../spread_probability_top30/%d.csv'%i) for i in range(1,5)],ignore_index=True)
spread_prob_df['from'] = spread_prob_df['from'].apply(lambda x: '%012d'%x)
spread_prob_df['to'] = spread_prob_df['to'].apply(lambda x: '%012d'%x)
spread_prob_df = spread_prob_df[['from', 'to', 'prob']].reset_index(drop = True)

In [3]:
import random

In [4]:
random.seed(42)
np.random.seed(42)

In [5]:
pop_data = pd.read_csv('../src_data/usa_population_revise.csv', dtype ={'GeoId':str, 'Population':np.int64})

In [6]:
pop_data

,GeoId,Population
0,010010201001,730
1,010010201002,1263
2,010010202001,835
3,010010202002,1124
4,010010203001,2774
...,...,...
220329,721537506011,883
220330,721537506012,2523
220331,721537506013,991
220332,721537506021,1577


In [7]:
starts = list(spread_prob_df['from'].unique())
random.shuffle(starts)
starts = starts[:5]

In [8]:
def init_counter():
    counter = {}
    total_infected_tracker = {}
    N = len(pop_data)
    for i in range(N):
        cbg = pop_data.loc[i,'GeoId']
        pop = int(pop_data.loc[i,'Population'])
        counter[cbg] = [pop, 0, 0] # S, I, R
        total_infected_tracker[cbg] = [pop, 0] # population, total infected
    return counter, total_infected_tracker

In [39]:
num_init_cases = 500
infectious_rate = [0.1,0.2,0.3,0.3,0.2,0.1,0.1]
infectious_day = len(infectious_rate)
spread_prob_grouped_df = spread_prob_df.groupby('from')
active_case = {}
counter,total_case_tracker = init_counter()
region_level_output_str = 'Day,GeoId,Susceptible,Infectious,Recovered\n'
case_level_output_str = 'Day,from_GeoId,from_case,to_GeoId,to_case\n'
simulation_day = 0
# random initiate spread in 5 regions
for s in starts:
    num_init_cases_refine = min(counter[s][0], num_init_cases)
    active_case[s] = [[0] * infectious_day, [0] * infectious_day] # [case/day, idx/day]
    active_case[s][0][0] = num_init_cases_refine
    counter[s][0] -= num_init_cases_refine
    counter[s][1] += num_init_cases_refine
    total_case_tracker[s][1] += num_init_cases_refine
    # region level output update
    # header: Day, GeoId, Susceptible, Infectious, Recovered
    region_level_output_str += '%d,%s,%d,%d,%d\n'%(simulation_day, s, counter[s][0], counter[s][1], counter[s][2])

    # case level output update
    # header: Day, from_GeoId, from_case, to_GeoId, to_case
    # simulation initialed: 000..0, -1
    simu_GeoId = '0' * len(s)
    for i in range(num_init_cases_refine):
        case_level_output_str += '%d,%s,%d,%s,%d\n'%(simulation_day,simu_GeoId, -1, s, i)

# save the outputs
# Todo

In [10]:
print(region_level_output_str)

Day,GeoId,Susceptible,Infectious,Recovered
0,250092524002,1511,500,0
0,361190111013,939,500,0
0,270530257013,679,500,0
0,360650228002,979,500,0
0,261635154001,156,500,0



In [11]:
def get_random_des(des_prob_df):
    rand_val = random.random()
    cumprob = des_prob_df['prob'].cumsum()
    for des, prob in zip(des_prob_df['to'], cumprob):
        if rand_val < prob: return des
    return des_prob_df.loc[-1,'to']

In [40]:
def next_day_new(simulation_day):
    region_level_output_str = ''
    case_level_output_str = ''
    recovered_cases = {}
    new_cases = {}
    for cbg in list(active_case.keys()):
        
        des_prob_df = spread_prob_grouped_df.get_group(cbg)
        cbg_active_case = active_case[cbg][0]
        cbg_case_idx_lower = active_case[cbg][1]
        cbg_active_case_num = 0
        for d in range(infectious_day-1, -1, -1):

            num_infectious = cbg_active_case[d]
            current_case_idx = cbg_case_idx_lower[d]
            cbg_active_case_num += num_infectious

            # collect recovered cases
            if d == infectious_day-1:
                if num_infectious != 0: recovered_cases[cbg] = num_infectious

            # update active case tracker
            if d != 0:
                cbg_active_case[d] = cbg_active_case[d-1]
                cbg_case_idx_lower[d] = cbg_case_idx_lower[d-1]
            else:
                cbg_active_case[d] = 0
                if cbg_active_case_num == 0: # remove the cbg from the active case tracker
                    del active_case[cbg]

            # Skip when no active case
            if num_infectious == 0: continue
                
            # calculate the number of expected new cases in infectious day 'd'
            expected_new_case = num_infectious * infectious_rate[d]

            new_case_by_day = {}
            actual_new_case = 0
            if expected_new_case < 30:
                if num_infectious < 30: # naive approach
                    for idx in range(num_infectious):
                        if random.random() < infectious_rate[d]:
                            des_cbg = get_random_des(des_prob_df)
                            reduce_prob = total_case_tracker[des_cbg][1]/total_case_tracker[des_cbg][0]
                            if counter[des_cbg][0] == 0 or random.random() < reduce_prob: continue # Assume spread to an already infected person
                            if des_cbg not in new_cases: new_cases[des_cbg] = 0
                            new_cases[des_cbg] += 1
                            total_case_tracker[des_cbg][1] += 1
                            counter[des_cbg][0] -= 1
                            counter[des_cbg][1] += 1

                            # update case level output
                            case_level_output_str += '%d,%s,%d,%s,%d\n'%(simulation_day, cbg, idx+current_case_idx, des_cbg, total_case_tracker[des_cbg][1]-1)

                else: # normal distribution on number of new cases, then naive approach for destinations
                    actual_new_case = np.random.normal(expected_new_case, expected_new_case * (1-infectious_rate[d]))
                    actual_new_case = int(max(0,actual_new_case))
                    failed_spread = 0
                    for _ in range(actual_new_case):
                        des_cbg = get_random_des(des_prob_df)
                        reduce_prob = total_case_tracker[des_cbg][1]/total_case_tracker[des_cbg][0]
                        if counter[des_cbg][0] == 0 or random.random() < reduce_prob: # Assume spread to an already infected person
                            failed_spread += 1
                            continue
                        if des_cbg not in new_case_by_day:
                            new_case_by_day[des_cbg] = 0
                        new_case_by_day[des_cbg] += 1
                        total_case_tracker[des_cbg][1] += 1
                        counter[des_cbg][0] -= 1
                        counter[des_cbg][1] += 1
                    actual_new_case -= failed_spread

            else: # calculate the expected number of new cases, then use normal/Poisson distribution for destinations
                for des_idx in des_prob_df.index:
                    des_cbg = des_prob_df.loc[des_idx,'to']
                    des_susceptible = counter[des_cbg][0]
                    if des_susceptible == 0: continue # No susceptible population

                    des_prob = des_prob_df.loc[des_idx,'prob']
                    reduce_prob = total_case_tracker[des_cbg][1]/total_case_tracker[des_cbg][0]
                    expected_new_case_in_des = expected_new_case * des_prob * (1-reduce_prob)

                    if expected_new_case_in_des < 5: # Poisson distribution
                        actual_new_case_in_des = np.random.poisson(expected_new_case_in_des)
                    else: # Normal distribution
                        actual_new_case_in_des = np.random.normal(expected_new_case_in_des, expected_new_case_in_des* (1-des_prob))
                        actual_new_case_in_des = int(max(0,actual_new_case_in_des))
                    actual_new_case_in_des = min(des_susceptible, actual_new_case_in_des)
                    if actual_new_case_in_des == 0: continue # No spread happens

                    actual_new_case += actual_new_case_in_des
                    if des_cbg not in new_case_by_day:
                        new_case_by_day[des_cbg] = 0
                    new_case_by_day[des_cbg] += actual_new_case_in_des
                    total_case_tracker[des_cbg][1] += actual_new_case_in_des
                    counter[des_cbg][0] -= actual_new_case_in_des
                    counter[des_cbg][1] += actual_new_case_in_des

            sidx = 0
            src_case_idx = np.random.choice(num_infectious, size = actual_new_case, replace = False) + current_case_idx if actual_new_case else None
            for des_cbg, des_case in new_case_by_day.items(): # Update new case counter
                if des_cbg not in new_cases: new_cases[des_cbg] = 0
                new_cases[des_cbg] += des_case
                # Update case level output
                if actual_new_case == 0: continue # Skip for naive approach
                des_start_idx = total_case_tracker[des_cbg][1] - des_case
                for des_idx in range(des_case):
                    case_level_output_str += '%d,%s,%d,%s,%d\n'%(simulation_day,cbg,src_case_idx[sidx],des_cbg,des_start_idx+des_idx)
                    sidx += 1

    # Spread finished
    # Update active case tracker
    for des_cbg, des_case in new_cases.items():
        if des_cbg not in active_case:
            active_case[des_cbg] = [[0] * infectious_day, [0] * infectious_day]
        active_case[des_cbg][0][0] = des_case
        active_case[des_cbg][1][0] = total_case_tracker[des_cbg][1] - des_case

        # Update region level output with new cases
        region_level_output_str += '%d,%s,%d,%d,%d\n'%(simulation_day, des_cbg, counter[des_cbg][0], counter[des_cbg][1], counter[des_cbg][2])

    # Update counter with recovered cases
    for cbg, case in recovered_cases.items():
        counter[cbg][1] -= case
        counter[cbg][2] += case

        # Update region level output with recovered cases
        region_level_output_str += '%d,%s,%d,%d,%d\n'%(simulation_day, cbg, counter[cbg][0], counter[cbg][1], counter[cbg][2])

    return region_level_output_str, case_level_output_str

In [41]:
active_case

{'250092524002': [[500, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]],
 '361190111013': [[500, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]],
 '270530257013': [[500, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]],
 '360650228002': [[500, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]],
 '261635154001': [[500, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]]}

In [42]:
r,c = next_day_new(1)
active_case

[500, 0, 0, 0, 0, 0, 0]
[500, 0, 0, 0, 0, 0, 0]
[500, 0, 0, 0, 0, 0, 0]
[500, 0, 0, 0, 0, 0, 0]
[500, 0, 0, 0, 0, 0, 0]


{'250092524002': [[1, 500, 0, 0, 0, 0, 0], [500, 0, 0, 0, 0, 0, 0]],
 '361190111013': [[0, 500, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]],
 '270530257013': [[0, 500, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]],
 '360650228002': [[3, 500, 0, 0, 0, 0, 0], [500, 0, 0, 0, 0, 0, 0]],
 '261635154001': [[0, 500, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]],
 '330151003012': [[3, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]],
 '250092501001': [[3, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]],
 '330151061022': [[5, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]],
 '330151004004': [[2, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]],
 '330151003011': [[3, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]],
 '330151061021': [[1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]],
 '250092503001': [[1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]],
 '330151061023': [[1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]],
 '330151002001': [[1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]],
 '330151003021': [[2, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]],
 '250092517

In [43]:
print(r)

1,330151003012,740,3,0
1,250092501001,2896,3,0
1,250092524002,1510,501,0
1,330151061022,2433,5,0
1,330151004004,1953,2,0
1,330151003011,2127,3,0
1,330151061021,3147,1,0
1,250092503001,2110,1,0
1,330151061023,1419,1,0
1,330151002001,1753,1,0
1,330151003021,2070,2,0
1,250092517002,1738,3,0
1,250092525023,1288,3,0
1,330151004003,1888,3,0
1,250092507001,1963,1,0
1,330151003013,1514,1,0
1,330151004001,1623,2,0
1,330151031003,2954,2,0
1,330151061013,1555,1,0
1,250092522021,2049,1,0
1,330151002003,1078,1,0
1,250092516002,2642,2,0
1,250092525011,2275,1,0
1,250092508001,1568,1,0
1,250092524001,1499,1,0
1,361190094002,983,5,0
1,361190094001,831,2,0
1,361190093001,503,4,0
1,361190093003,1360,3,0
1,361190094003,2044,1,0
1,361190050023,1795,4,0
1,361190050011,2432,4,0
1,361190111011,3539,1,0
1,361190093002,1097,1,0
1,361190110002,1429,1,0
1,361190086024,1546,2,0
1,361190106002,1467,2,0
1,361190050022,1571,1,0
1,361190090002,2213,2,0
1,361190121013,2259,2,0
1,361190106001,1410,3,0
1,360610092001,147

In [44]:
print(c)

1,250092524002,211,330151003012,0
1,250092524002,434,330151003012,1
1,250092524002,38,330151003012,2
1,250092524002,149,250092501001,0
1,250092524002,118,250092501001,1
1,250092524002,23,250092501001,2
1,250092524002,316,250092524002,500
1,250092524002,245,330151061022,0
1,250092524002,111,330151061022,1
1,250092524002,215,330151061022,2
1,250092524002,420,330151061022,3
1,250092524002,450,330151061022,4
1,250092524002,78,330151004004,0
1,250092524002,323,330151004004,1
1,250092524002,181,330151003011,0
1,250092524002,69,330151003011,1
1,250092524002,387,330151003011,2
1,250092524002,489,330151061021,0
1,250092524002,232,250092503001,0
1,250092524002,37,330151061023,0
1,250092524002,276,330151002001,0
1,250092524002,91,330151003021,0
1,250092524002,159,330151003021,1
1,250092524002,308,250092517002,0
1,250092524002,129,250092517002,1
1,250092524002,413,250092517002,2
1,250092524002,255,250092525023,0
1,250092524002,380,250092525023,1
1,250092524002,352,250092525023,2
1,250092524002,155

In [45]:
for i in range(2,7):
    r,c = next_day_new(i)
active_case

[1, 500, 0, 0, 0, 0, 0]
[0, 500, 0, 0, 0, 0, 0]
[0, 500, 0, 0, 0, 0, 0]
[3, 500, 0, 0, 0, 0, 0]
[0, 500, 0, 0, 0, 0, 0]
[3, 0, 0, 0, 0, 0, 0]
[3, 0, 0, 0, 0, 0, 0]
[5, 0, 0, 0, 0, 0, 0]
[2, 0, 0, 0, 0, 0, 0]
[3, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[2, 0, 0, 0, 0, 0, 0]
[3, 0, 0, 0, 0, 0, 0]
[3, 0, 0, 0, 0, 0, 0]
[3, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[2, 0, 0, 0, 0, 0, 0]
[2, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[2, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[5, 0, 0, 0, 0, 0, 0]
[2, 0, 0, 0, 0, 0, 0]
[4, 0, 0, 0, 0, 0, 0]
[3, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[4, 0, 0, 0, 0, 0, 0]
[4, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[2, 0, 0, 0, 0, 0, 0]
[2, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0]
[2, 0, 0, 0, 0, 0, 0]
[2, 0, 0, 0, 0, 0, 0]
[3, 0, 0, 0, 0, 0, 0]


{'250092524002': [[1, 5, 2, 13, 3, 1, 500], [524, 519, 517, 504, 501, 500, 0]],
 '361190111013': [[0, 0, 0, 0, 0, 0, 500], [0, 0, 0, 0, 0, 0, 0]],
 '270530257013': [[2, 0, 1, 5, 4, 0, 500], [510, 509, 509, 504, 500, 0, 0]],
 '360650228002': [[5, 5, 3, 4, 6, 3, 500], [521, 516, 513, 509, 503, 500, 0]],
 '261635154001': [[0, 0, 0, 0, 0, 0, 500], [0, 0, 0, 0, 0, 0, 0]],
 '330151003012': [[16, 7, 4, 2, 27, 3, 0], [43, 36, 32, 30, 3, 0, 0]],
 '250092501001': [[12, 10, 9, 12, 17, 3, 0], [51, 41, 32, 20, 3, 0, 0]],
 '330151061022': [[7, 7, 3, 1, 5, 5, 0], [21, 14, 11, 10, 5, 0, 0]],
 '330151004004': [[7, 7, 7, 1, 4, 2, 0], [21, 14, 7, 6, 2, 0, 0]],
 '330151003011': [[10, 9, 11, 10, 5, 3, 0], [38, 29, 18, 8, 3, 0, 0]],
 '330151061021': [[8, 5, 4, 11, 3, 1, 0], [24, 19, 15, 4, 1, 0, 0]],
 '250092503001': [[4, 7, 5, 6, 2, 1, 0], [21, 14, 9, 3, 1, 0, 0]],
 '330151061023': [[7, 6, 5, 4, 0, 1, 0], [16, 10, 5, 1, 0, 0, 0]],
 '330151002001': [[3, 6, 5, 4, 3, 1, 0], [19, 13, 8, 4, 1, 0, 0]],
 '3301510

In [46]:
print(r)

6,330151003012,684,59,0
6,250092501001,2836,63,0
6,250092524002,1486,525,0
6,330151061022,2410,28,0
6,330151004004,1927,28,0
6,330151003011,2082,48,0
6,330151061021,3116,32,0
6,330151001001,2753,26,0
6,250092503001,2086,25,0
6,330151061011,2865,22,0
6,330151061023,1397,23,0
6,330151002001,1732,22,0
6,330151003021,2044,28,0
6,250092525023,1270,21,0
6,250092507001,1940,24,0
6,250092502001,2387,27,0
6,330151003013,1491,24,0
6,330151004001,1605,20,0
6,250092522021,2028,22,0
6,250092516002,2623,21,0
6,250092525011,2261,15,0
6,250092508001,1553,16,0
6,250092524001,1487,13,0
6,250092522012,2017,9,0
6,361190094002,939,49,0
6,361190094001,790,43,0
6,361190093001,435,72,0
6,361190093003,1312,51,0
6,361190094003,2004,41,0
6,361190050023,1766,33,0
6,361190050011,2406,30,0
6,361190111011,3523,17,0
6,361190093002,1074,24,0
6,361190110002,1412,18,0
6,361190111022,1416,15,0
6,361190106002,1450,19,0
6,361190050022,1554,18,0
6,361190090002,2197,18,0
6,361190109011,2143,17,0
6,361190106001,1391,22,0
6,36

In [47]:
r,c = next_day_new(7)
print(r)

[1, 5, 2, 13, 3, 1, 500]
[0, 0, 0, 0, 0, 0, 500]
[2, 0, 1, 5, 4, 0, 500]
[5, 5, 3, 4, 6, 3, 500]
[0, 0, 0, 0, 0, 0, 500]
[16, 7, 4, 2, 27, 3, 0]
[12, 10, 9, 12, 17, 3, 0]
[7, 7, 3, 1, 5, 5, 0]
[7, 7, 7, 1, 4, 2, 0]
[10, 9, 11, 10, 5, 3, 0]
[8, 5, 4, 11, 3, 1, 0]
[4, 7, 5, 6, 2, 1, 0]
[7, 6, 5, 4, 0, 1, 0]
[3, 6, 5, 4, 3, 1, 0]
[4, 6, 6, 7, 3, 2, 0]
[5, 4, 7, 3, 0, 3, 0]
[2, 4, 4, 4, 4, 3, 0]
[2, 5, 6, 8, 3, 3, 0]
[6, 4, 3, 5, 5, 1, 0]
[3, 3, 6, 9, 2, 1, 0]
[4, 2, 7, 3, 2, 2, 0]
[3, 4, 11, 5, 1, 2, 0]
[6, 2, 8, 6, 5, 1, 0]
[6, 4, 5, 3, 3, 1, 0]
[1, 2, 5, 3, 0, 1, 0]
[4, 4, 4, 3, 4, 2, 0]
[5, 2, 5, 1, 1, 1, 0]
[3, 5, 1, 5, 1, 1, 0]
[2, 2, 5, 2, 1, 1, 0]
[5, 24, 5, 5, 5, 5, 0]
[10, 6, 14, 5, 6, 2, 0]
[7, 12, 2, 32, 15, 4, 0]
[6, 4, 15, 21, 2, 3, 0]
[3, 6, 13, 12, 6, 1, 0]
[4, 5, 18, 0, 2, 4, 0]
[3, 10, 0, 11, 2, 4, 0]
[2, 2, 1, 5, 6, 1, 0]
[4, 4, 5, 5, 5, 1, 0]
[4, 3, 6, 3, 1, 1, 0]
[2, 4, 2, 6, 4, 2, 0]
[3, 8, 3, 1, 2, 2, 0]
[1, 3, 5, 4, 4, 1, 0]
[2, 3, 5, 3, 3, 2, 0]
[2, 1, 3, 5, 2, 2, 